#### Ingestion

In [0]:
%run "../utils/configs/functions"

In [0]:
%run "../utils/configs/paths"

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, TimestampType, DateType

In [0]:
sch = StructType(
    [
        StructField("raceid", IntegerType(), False),
        StructField("year", IntegerType(), False),
        StructField("round", IntegerType(), True),
        StructField("circuitId", IntegerType(), True),
        StructField("name", StringType(), True),
        StructField("date", StringType(), True),
        StructField("time", StringType(), True),
        StructField("url", StringType(), True),
    ]
)

In [0]:
output = filename("2021-03-21")
df = spark.read.option("header",True).schema(sch).csv(f"{raw_path}/{output}/races.csv")

##### Transformations

In [0]:
from pyspark.sql.functions import col, current_timestamp , to_timestamp, concat, lit

In [0]:
semi_final_df = df.withColumn("race_timestamp", to_timestamp(concat(col("date"), lit(" "), col("time")), "yyyy-MM-dd HH:mm:ss")).select("raceid","year","round","circuitId","name","race_timestamp").withColumnRenamed("raceId", "race_id").withColumnRenamed("circuitId", "circuit_id").withColumnRenamed("year","race_year")
final_df = current_time_ingestion(semi_final_df)

#### Writing as Spark Tables as parquet format

In [0]:
final_df.write.mode("overwrite").partitionBy("race_year").format("parquet").saveAsTable("f1_processed.races")